In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import shutil
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
basic_info = load_data("产品目录.xlsx")
basic_info

,大类策略,细分策略,基金代码,基金名称,基准代码,基准名称,最大回撤阈值
0,主观CTA,复合,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.05
1,主观CTA,复合,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10
2,主观CTA,趋势,SGC034,济海实恒对冲二号,NH0100.NHF,南华商品指数,-0.10
3,主观CTA,复合,ST6571,星恒哲萌1号,NH0100.NHF,南华商品指数,-0.05
4,量化CTA,截面,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
5,量化CTA,趋势,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.03
6,量化CTA,趋势,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.05
7,量化CTA,趋势,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.05
8,量化CTA,趋势,SN3221,宏锡量化CTA7号,NH0100.NHF,南华商品指数,-0.05
9,量化CTA,复合,SGA623,会世元丰CTA1号,NH0100.NHF,南华商品指数,-0.10


In [3]:
# 删除html文件
def delete_html_files(directory):
    # 遍历目录及其子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否以.html 结尾
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                try:
                    # 删除文件
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

directory_to_clean = "docs"
delete_html_files(directory_to_clean)

Deleted: docs\index.html
Deleted: docs\主观CTA\20180907_20241220_草本优益1号_nav_analysis.html
Deleted: docs\主观CTA\20190307_20241220_济海实恒对冲二号_nav_analysis.html
Deleted: docs\主观CTA\20201104_20241220_草本致远1号_nav_analysis.html
Deleted: docs\主观CTA\20210414_20241220_星恒哲萌1号_nav_analysis.html
Deleted: docs\套利\20170512_20241220_远澜翠柏1号_nav_analysis.html
Deleted: docs\量化CTA\20151104_20241220_远澜云杉_nav_analysis.html
Deleted: docs\量化CTA\20170217_20241220_宏锡量化CTA7号_nav_analysis.html
Deleted: docs\量化CTA\20170301_20241220_均成CTA1号_nav_analysis.html
Deleted: docs\量化CTA\20180730_20241213_会世元丰CTA1号_nav_analysis.html
Deleted: docs\量化CTA\20200910_20241220_弘源多元化CTA_nav_analysis.html
Deleted: docs\量化CTA\20221021_20241220_众壹资产量合兴成一号_nav_analysis.html
Deleted: docs\量化CTA\20230106_20241220_量派CTA七号_nav_analysis.html
Deleted: docs\量化CTA\20240105_20241220_千衍九凌1号_nav_analysis.html


In [4]:
# 生成新的html文件
files_list_series = pd.Series([i.stem for i in Path("./data").glob("*.xlsx")])
for row in basic_info.itertuples(index=False, name=None):
    nav_df_path = Path("data").joinpath(f"{files_list_series[files_list_series.str.contains(row[3])].item()}.xlsx")
    demo = NavResearch(nav_df_path,row[0],row[2],row[3],row[4],row[5],row[6])
    demo.get_data()
    demo.get_analysis_table()
    demo.get_html()

草本优益1号无法推断频率,自动转为周度
济海实恒对冲二号无法推断频率,自动转为周度
均成CTA1号无法推断频率,自动转为周度
弘源多元化CTA无法推断频率,自动转为周度
宏锡量化CTA7号无法推断频率,自动转为周度
会世元丰CTA1号无法推断频率,自动转为周度
远澜翠柏1号无法推断频率,自动转为周度


In [6]:
# 生成新的index.html文件
def generate_index_html(folder_path: Path):
    # 获取目录及其子目录中的所有 HTML 文件
    html_files = folder_path.rglob("*.html")
    print(html_files)

    # 创建一个新的 index.html 文件
    with open(folder_path.joinpath("index.html"), "w", encoding="utf-8") as f:
        f.write(
            """<html>
<head>
<meta charset="UTF-8">
<title>Value over Time</title>
<style>
    body {
        font-family: Arial, sans-serif;
        font-size: 20px;
        color: blue;
    }
    h2 {
        font-size: 25px;
        color: blue;

    }
    a {
        text-decoration: none;
    }
</style>
</head>
<body>"""
        )
        # 创建一个集合来存储已经处理过的文件夹，避免重复创建链接
        processed_folders = set()
        # 为每个 HTML 文件创建一个链接
        for html_file in html_files:
            if html_file.name == "index.html":
                continue
            relative_path = html_file.relative_to(folder_path)
            # 如果文件夹还没有被处理过，为文件夹创建一个链接
            if html_file.parent not in processed_folders:
                f.write(f"<h2><a>{html_file.parent.name}</a></h2>\n")
                processed_folders.add(html_file.parent)

            # 创建HTML文件的链接
            cleaned_name = relative_path.name.replace("_nav_analysis", "")
            f.write(f'<a href="{relative_path}" target="_blank">{cleaned_name}</a><br/>\n')

        f.write("</body>\n</html>")

# 使用函数
generate_index_html(Path("docs"))

<generator object Path.rglob at 0x000001DF6102A570>
